# Human numbers

In [1]:
from fastai.text import *

In [2]:
bs=64

## Data

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [9]:
!pip install -q git+https://github.com/fastai/fastprogress --upgrade
!pip install -q git+https://github.com/fastai/fastcore --upgrade
!pip install -q git+https://github.com/fastai/fastai2 --upgrade

ERROR: torchvision 0.5.0 has requirement torch==1.4.0, but you'll have torch 1.3.1 which is incompatible.


In [13]:
?progress_bar

In [32]:
path = untar_data(URLs.HUMAN_NUMBERS)
path.ls()

[PosixPath('/home/shanmugam/.fastai/data/human_numbers/train.txt'),
 PosixPath('/home/shanmugam/.fastai/data/human_numbers/valid.txt')]

In [33]:
def readnums(d): return [', '.join(o.strip() for o in open(path/d).readlines())]

In [34]:
train_txt = readnums('train.txt'); train_txt[0][:80]

'one, two, three, four, five, six, seven, eight, nine, ten, eleven, twelve, thirt'

In [35]:

valid_txt = readnums('valid.txt'); valid_txt[0][-80:]

' nine thousand nine hundred ninety eight, nine thousand nine hundred ninety nine'

In [36]:
train = TextList(train_txt, path=path)
valid = TextList(valid_txt, path=path)

src = ItemLists(path=path, train=train, valid=valid).label_for_lm()
data = src.databunch(bs=bs, bptt=4)

In [37]:
train[0].text[:80]

'xxbos one , two , three , four , five , six , seven , eight , nine , ten , eleve'

In [38]:
len(data.valid_ds[0][0].data)

13017

In [39]:
data.bptt, len(data.valid_dl)

(4, 51)

In [40]:
13017/70/bs

2.905580357142857

In [41]:
it = iter(data.valid_dl)
x1,y1 = next(it)
x2,y2 = next(it)
x3,y3 = next(it)
it.close()

In [42]:
x1.numel()+x2.numel()+x3.numel()

768

In [43]:
x1.shape,y1.shape

(torch.Size([64, 4]), torch.Size([64, 4]))

In [44]:
x2.shape,y2.shape

(torch.Size([64, 4]), torch.Size([64, 4]))

In [45]:
x1[:,3]

tensor([12, 23,  9, 14, 11, 27, 19, 13,  9,  9, 10, 19, 10, 17, 11, 27, 10, 16,
         9, 17, 10, 19, 10, 21, 11, 27, 19, 19, 20,  9, 23, 11, 20, 20, 18, 10,
        20, 10, 32, 11, 26, 14, 14, 15, 14, 10,  9, 10, 11, 11, 26, 20, 17, 18,
        17, 10,  9, 10, 21, 11, 26, 10, 10,  9], device='cuda:0')

In [46]:
y1[:,3]

tensor([ 9, 17, 19,  9, 12, 13, 11, 10, 19, 19, 22, 11, 28,  9, 15, 16, 36, 10,
        19, 10, 22, 11, 28, 20, 18, 19, 11, 10,  9, 19, 12, 20, 11, 11,  9, 21,
        11, 27,  9, 13, 14,  9, 10,  9, 10, 21, 20, 27, 16, 16, 17, 11, 10,  9,
        10, 21, 20, 27,  9, 19, 20, 29, 23, 20], device='cuda:0')

In [47]:
v = data.valid_ds.vocab

In [48]:
v.textify(x1[0])

'xxbos eight thousand one'

In [49]:
v.textify(y1[0])

'eight thousand one ,'

In [50]:
v.textify(x2[0])

', eight thousand two'

In [51]:
v.textify(x3[0])

', eight thousand three'

In [ ]:
v.textify(x1[1])

', eight thousand forty six , eight thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine ,'

In [ ]:
v.textify(x2[1])

'eight thousand sixty , eight thousand sixty one , eight thousand sixty two , eight thousand sixty three , eight thousand sixty four , eight thousand sixty five , eight thousand sixty six , eight thousand sixty seven , eight thousand sixty eight , eight thousand sixty nine , eight thousand seventy , eight thousand seventy one , eight thousand seventy two , eight thousand seventy three , eight thousand'

In [ ]:
v.textify(x3[1])

'seventy four , eight thousand seventy five , eight thousand seventy six , eight thousand seventy seven , eight thousand seventy eight , eight thousand seventy nine , eight thousand eighty , eight thousand eighty one , eight thousand eighty two , eight thousand eighty three , eight thousand eighty four , eight thousand eighty five , eight thousand eighty six , eight thousand eighty seven , eight thousand eighty'

In [ ]:
v.textify(x3[-1])

'ninety , nine thousand nine hundred ninety one , nine thousand nine hundred ninety two , nine thousand nine hundred ninety three , nine thousand nine hundred ninety four , nine thousand nine hundred ninety five , nine thousand nine hundred ninety six , nine thousand nine hundred ninety seven , nine thousand nine hundred ninety eight , nine thousand nine hundred ninety nine xxbos eight thousand one , eight'

In [24]:
data.show_batch(ds_type=DatasetType.Valid)

idx,text
0,"thousand forty seven , eight thousand forty eight , eight thousand forty nine , eight thousand fifty , eight thousand fifty one , eight thousand fifty two , eight thousand fifty three , eight thousand fifty four , eight thousand fifty five , eight thousand fifty six , eight thousand fifty seven , eight thousand fifty eight , eight thousand fifty nine , eight thousand sixty , eight thousand sixty"
1,"eight , eight thousand eighty nine , eight thousand ninety , eight thousand ninety one , eight thousand ninety two , eight thousand ninety three , eight thousand ninety four , eight thousand ninety five , eight thousand ninety six , eight thousand ninety seven , eight thousand ninety eight , eight thousand ninety nine , eight thousand one hundred , eight thousand one hundred one , eight thousand one"
2,"thousand one hundred twenty four , eight thousand one hundred twenty five , eight thousand one hundred twenty six , eight thousand one hundred twenty seven , eight thousand one hundred twenty eight , eight thousand one hundred twenty nine , eight thousand one hundred thirty , eight thousand one hundred thirty one , eight thousand one hundred thirty two , eight thousand one hundred thirty three , eight thousand"
3,"three , eight thousand one hundred fifty four , eight thousand one hundred fifty five , eight thousand one hundred fifty six , eight thousand one hundred fifty seven , eight thousand one hundred fifty eight , eight thousand one hundred fifty nine , eight thousand one hundred sixty , eight thousand one hundred sixty one , eight thousand one hundred sixty two , eight thousand one hundred sixty three"
4,"thousand one hundred eighty three , eight thousand one hundred eighty four , eight thousand one hundred eighty five , eight thousand one hundred eighty six , eight thousand one hundred eighty seven , eight thousand one hundred eighty eight , eight thousand one hundred eighty nine , eight thousand one hundred ninety , eight thousand one hundred ninety one , eight thousand one hundred ninety two , eight thousand"


## Single fully connected model

In [27]:
data = src.databunch(bs=bs, bptt=3)

In [28]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 3]), torch.Size([64, 3]))

In [31]:
v.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 ',',
 'hundred',
 'thousand',
 'one',
 'two',
 'three',
 'four',
 'five',
 'six',
 'seven',
 'eight',
 'nine',
 'twenty',
 'thirty',
 'forty',
 'fifty',
 'sixty',
 'seventy',
 'eighty',
 'ninety',
 'ten',
 'eleven',
 'twelve',
 'thirteen',
 'fourteen',
 'fifteen',
 'sixteen',
 'seventeen',
 'eighteen',
 'nineteen',
 'xxfake']

In [52]:
nv = len(v.itos); nv

40

In [53]:
nh=64

In [ ]:
def loss4(input,target): return F.cross_entropy(input, target[:,-1])
def acc4 (input,target): return accuracy(input, target[:,-1])

In [ ]:
class Model0(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = self.bn(F.relu(self.h_h(self.i_h(x[:,0]))))
        if x.shape[1]>1:
            h = h + self.i_h(x[:,1])
            h = self.bn(F.relu(self.h_h(h)))
        if x.shape[1]>2:
            h = h + self.i_h(x[:,2])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [ ]:
learn = Learner(data, Model0(), loss_func=loss4, metrics=acc4)

In [ ]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4
1,3.596286,3.588869,0.046645
2,3.086100,3.205763,0.274816
3,2.494411,2.749365,0.392004
4,2.144753,2.463537,0.415671
5,2.010915,2.352887,0.409237
6,1.983992,2.336967,0.408778


## Same thing with a loop

In [ ]:
class Model1(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)  # green arrow
        self.h_h = nn.Linear(nh,nh)     # brown arrow
        self.h_o = nn.Linear(nh,nv)     # blue arrow
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = self.bn(F.relu(self.h_h(h)))
        return self.h_o(h)

In [ ]:
learn = Learner(data, Model1(), loss_func=loss4, metrics=acc4)

In [ ]:
learn.fit_one_cycle(6, 1e-4)

epoch,train_loss,valid_loss,acc4
1,3.493525,3.420231,0.156250
2,2.987600,2.937893,0.376149
3,2.440199,2.477995,0.388787
4,2.132837,2.256569,0.391774
5,2.011305,2.181337,0.392923
6,1.985913,2.170874,0.393153


## Multi fully connected model

In [ ]:
data = src.databunch(bs=bs, bptt=20)

In [ ]:
x,y = data.one_batch()
x.shape,y.shape

(torch.Size([64, 20]), torch.Size([64, 20]))

In [ ]:
class Model2(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        
    def forward(self, x):
        h = torch.zeros(x.shape[0], nh).to(device=x.device)
        res = []
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.h_o(self.bn(h)))
        return torch.stack(res, dim=1)

In [ ]:
learn = Learner(data, Model2(), metrics=accuracy)

In [ ]:
learn.fit_one_cycle(10, 1e-4, pct_start=0.1)

epoch,train_loss,valid_loss,accuracy
1,3.639285,3.709278,0.058949
2,3.551151,3.565677,0.151776
3,3.439908,3.431850,0.207741
4,3.323083,3.314237,0.283949
5,3.213422,3.219906,0.321662
6,3.119673,3.151162,0.336790
7,3.046645,3.106630,0.341690
8,2.995379,3.082552,0.346662
9,2.963800,3.073327,0.349645
10,2.947312,3.071951,0.349787


## Maintain state

In [ ]:
class Model3(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.h_h = nn.Linear(nh,nh)
        self.h_o = nn.Linear(nh,nv)
        self.bn = nn.BatchNorm1d(nh)
        self.h = torch.zeros(bs, nh).cuda()
        
    def forward(self, x):
        res = []
        h = self.h
        for i in range(x.shape[1]):
            h = h + self.i_h(x[:,i])
            h = F.relu(self.h_h(h))
            res.append(self.bn(h))
        self.h = h.detach()
        res = torch.stack(res, dim=1)
        res = self.h_o(res)
        return res

In [ ]:
learn = Learner(data, Model3(), metrics=accuracy)

In [ ]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy
1,3.598183,3.556362,0.050710
2,3.274616,2.975699,0.401634
3,2.624206,2.036894,0.467330
4,2.022702,1.956439,0.316193
5,1.681813,1.934952,0.336861
6,1.453007,1.948201,0.351349
7,1.276971,2.005776,0.368679
8,1.138499,2.081261,0.360156
9,1.029217,2.145853,0.360795
10,0.939949,2.215388,0.372230


## nn.RNN

In [ ]:
class Model4(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.RNN(nh,nh, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(1, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        self.h = h.detach()
        return self.h_o(self.bn(res))

In [ ]:
learn = Learner(data, Model4(), metrics=accuracy)

In [ ]:
learn.fit_one_cycle(20, 3e-3)

epoch,train_loss,valid_loss,accuracy
1,3.451432,3.268344,0.224148
2,2.974938,2.456569,0.466051
3,2.316732,1.946969,0.465625
4,1.866151,1.991952,0.314702
5,1.618516,1.802403,0.437216
6,1.411517,1.731107,0.436293
7,1.171916,1.655979,0.504048
8,0.965887,1.579963,0.522088
9,0.797046,1.479819,0.565057
10,0.659378,1.487831,0.579048


## 2-layer GRU

In [137]:
bs

64

In [144]:
h=torch.zeros(2, bs, nh)
h[1].shape

IndexError: index 2 is out of bounds for dimension 0 with size 2

x = torch.Size([64, 4])
self.h =torch.Size([2, 64, 64]), 
self.i_h(x) = torch.Size([64, 4, 64]),
res = torch.Size([64, 4, 64]),
h = torch.Size([2, 64, 64])
self.h_o(self.bn(res)) = torch.Size([64, 4, 40])

In [133]:
class Model5(nn.Module):
    def __init__(self):
        super().__init__()
        self.i_h = nn.Embedding(nv,nh)
        self.rnn = nn.GRU(nh, nh, 2, batch_first=True)
        self.h_o = nn.Linear(nh,nv)
        self.bn = BatchNorm1dFlat(nh)
        self.h = torch.zeros(2, bs, nh).cuda()
        
    def forward(self, x):
        res,h = self.rnn(self.i_h(x), self.h)
        
        self.h = h.detach()
        print(self.bn(res))
#         print((self.h_o(self.bn(res))).shape)
        return self.h_o(self.bn(res))

In [134]:
learn = Learner(data, Model5(), metrics=accuracy)

In [135]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.235391,2.286150,0.467754,00:03


tensor([[[-2.1212, -1.0689, -1.3061,  ..., -2.5178,  1.3680,  0.4953],
         [-1.8123,  0.1419, -1.2986,  ..., -2.4947,  2.0488,  1.6271],
         [-0.7081,  0.6138, -2.1294,  ..., -3.0613,  2.6940,  1.0781],
         [-0.5311,  0.6393, -2.4957,  ..., -1.5763,  1.8799,  0.9925]],

        [[ 0.9027, -0.7658,  0.3327,  ...,  0.7479, -0.9128,  0.5831],
         [ 1.1771, -0.0645,  0.3728,  ...,  1.4729, -2.1330, -0.3808],
         [ 1.3699,  0.6974, -0.9998,  ..., -0.3698, -0.4919, -1.3871],
         [ 0.8623,  0.9059, -1.7050,  ..., -0.0049, -0.2974, -0.7975]],

        [[-1.1025, -1.0666,  0.5119,  ..., -0.1027,  0.0476, -0.1877],
         [-0.0757,  0.0668, -0.6189,  ..., -1.0162,  1.5167, -1.2996],
         [-0.5594, -0.1343,  0.1064,  ..., -1.2503,  1.3156, -2.3226],
         [ 1.4556,  0.7659,  0.5112,  ..., -0.4186,  0.2582, -2.1255]],

        ...,

        [[ 0.9027, -0.7658,  0.3327,  ...,  0.7479, -0.9128,  0.5831],
         [ 1.5986, -0.5148,  0.3886,  ...,  0.4080, -1.34

tensor([[[-0.5100, -0.4309, -3.3626,  ..., -0.1802, -1.3843, -1.4498],
         [-0.6519,  0.8357, -2.6987,  ...,  0.8990, -2.2694, -0.7881],
         [-0.4583,  1.0086, -3.0195,  ..., -0.0724, -1.4115, -0.8474],
         [-0.7921, -1.8413, -3.1801,  ..., -1.2866, -0.7760, -0.2623]],

        [[ 2.2967, -0.4711, -1.2576,  ...,  1.2513,  0.6094,  0.1765],
         [ 2.0625,  0.0907, -1.0112,  ...,  1.2323,  0.2299,  0.8247],
         [ 1.7315,  0.3917, -1.7264,  ..., -0.5808,  1.1871,  0.1560],
         [ 0.6971,  0.2438, -1.9751,  ..., -0.1438,  0.5635,  0.3290]],

        [[-0.6296, -1.5882, -1.8660,  ...,  1.2527, -1.1552, -0.9644],
         [-0.2305, -1.1143, -2.9213,  ...,  0.6399, -0.3948, -1.1876],
         [-1.0811, -1.4073, -1.9694,  ...,  0.4280, -0.6254, -1.3562],
         [ 1.2267, -0.6638, -1.1039,  ...,  1.0659, -0.7267, -0.7054]],

        ...,

        [[-0.3140,  0.2268,  0.3457,  ..., -1.0540, -1.2927, -0.2922],
         [ 1.4654,  0.2914,  0.5528,  ..., -0.2297, -1.04

tensor([[[-0.7859,  1.0919, -2.1373,  ...,  0.9129, -1.0279, -1.7032],
         [-0.4512,  0.5763, -1.5817,  ...,  1.3056, -1.1773, -1.9525],
         [-0.2207, -0.0090, -0.3677,  ...,  1.5681, -1.0941, -1.8096],
         [-0.2470,  0.6989, -1.5961,  ...,  1.0966, -0.5491, -2.0656]],

        [[ 0.4571, -0.3978, -1.6679,  ...,  1.0053, -0.6068, -1.1191],
         [-0.3435, -0.1827, -1.9517,  ...,  1.0567, -1.5340, -0.7703],
         [ 1.4248, -0.6845, -1.2195,  ...,  1.4761, -0.3597,  0.1756],
         [ 1.7337, -1.1810, -0.2517,  ...,  1.4200, -0.7427, -0.7665]],

        [[ 1.1155, -0.9668, -0.1595,  ...,  0.9939,  0.7774, -0.5496],
         [ 0.8528, -1.2918,  0.0368,  ...,  0.4081,  0.1351, -0.4130],
         [ 0.4855, -1.5430, -0.1021,  ...,  0.3488, -0.5082, -1.0850],
         [ 0.0943, -0.5846, -1.8765,  ..., -0.6567, -0.1533, -1.7739]],

        ...,

        [[ 0.5641, -0.7464, -1.0734,  ...,  0.7600, -0.2799, -1.5021],
         [ 0.1025,  0.2051, -1.0759,  ...,  0.4716, -2.40

tensor([[[-1.3556,  0.6837, -2.1498,  ..., -0.8226, -0.6335, -0.7648],
         [-1.7353,  0.4731, -1.7325,  ..., -0.8972, -0.4966, -0.6242],
         [-1.8248,  0.3085, -1.3997,  ..., -0.5775, -0.8064, -0.3084],
         [-1.7045,  0.7511, -2.2148,  ..., -1.3231, -0.7678, -0.8783]],

        [[ 0.2465, -0.2292, -1.3480,  ...,  0.1809, -0.4698, -1.9238],
         [-0.4313,  0.2577, -1.6250,  ...,  0.2179, -1.5048, -1.6062],
         [ 1.1697, -0.4208, -1.0820,  ...,  1.0237,  0.3598,  0.0635],
         [ 1.5374, -1.0293,  0.0810,  ...,  0.8804,  0.0327, -0.6886]],

        [[ 1.2241, -0.9489, -0.9026,  ...,  2.0207,  0.6026,  0.1303],
         [ 0.9670, -1.2077,  0.2208,  ...,  1.9322,  0.6713,  0.0036],
         [ 0.2739, -0.1131, -1.3241,  ...,  0.8095,  0.3457, -1.1598],
         [-0.6668, -0.0640, -0.8103,  ...,  0.7456, -0.8228, -1.5633]],

        ...,

        [[-0.2660, -0.4589, -1.6269,  ...,  0.1043, -0.4365, -1.1361],
         [-0.4994,  0.1846, -1.4014,  ..., -0.0297, -2.44

...

tensor([[[-5.5964e-02, -5.0910e-02, -9.8257e-01,  ..., -5.8290e-01,
           1.3972e+00,  4.4826e-01],
         [-1.4809e+00,  6.0698e-01, -1.0108e+00,  ...,  1.1104e+00,
          -1.2365e+00,  5.8004e-01],
         [ 1.6993e+00,  5.8549e-01, -5.9916e-01,  ..., -3.4939e-01,
           5.9795e-01,  2.7329e+00],
         [-7.2278e-01,  4.2575e-01, -6.6597e-01,  ..., -1.4921e+00,
          -1.8869e+00, -9.0105e-01]],

        [[ 1.8463e+00,  5.8593e-01, -4.7751e-01,  ..., -5.8344e-01,
           5.7649e-01,  2.7728e+00],
         [-5.1730e-01,  4.2954e-01, -4.4963e-01,  ..., -1.8163e+00,
          -1.2106e+00, -3.7424e-02],
         [-6.8887e-01, -1.5446e+00,  8.5637e-01,  ...,  5.2332e-02,
          -1.5066e+00, -5.3538e-01],
         [-4.2632e-01,  1.0582e-01, -9.3277e-01,  ..., -5.7907e-01,
           1.3899e+00,  1.4090e+00]],

        [[-8.8106e-01, -2.6373e-01,  3.3895e+00,  ..., -2.2140e+00,
          -1.4421e+00,  4.6872e-01],
         [ 7.9751e-01, -2.7616e-01,  8.4473e-01,  .

tensor([[[-1.4039,  0.5789, -1.0054,  ...,  1.2048, -1.3922,  0.9219],
         [ 1.9328,  0.5573, -0.5421,  ..., -0.4612,  0.6171,  2.6155],
         [-0.6142,  0.4043, -0.6122,  ..., -1.7713, -1.7352, -1.1381],
         [-0.7809, -1.6581,  1.6337,  ...,  0.2776, -0.9361, -1.0039]],

        [[-0.5840,  0.4307, -0.4003,  ..., -1.9687, -1.4554,  0.0504],
         [-0.8175, -0.9600,  0.6569,  ..., -0.5117, -1.1728, -0.3644],
         [ 0.0232,  0.1057, -1.0078,  ..., -0.9467,  1.3837,  1.2135],
         [-1.4706,  0.6166, -0.9844,  ...,  0.9552, -1.7517,  1.2076]],

        [[-1.4360,  0.6222, -0.6926,  ...,  0.7015,  0.1085,  0.8852],
         [-0.3004,  0.5948, -0.1489,  ..., -2.1307,  0.8707,  1.1935],
         [-0.7748, -0.3033,  4.0016,  ..., -2.5828, -1.5050,  0.3968],
         [ 0.9702, -0.3142,  1.1647,  ..., -0.2780,  1.1671,  2.4199]],

        ...,

        [[ 1.0053,  0.6840, -0.5764,  ..., -3.0589,  0.7685, -0.6300],
         [-0.3344,  0.5744,  1.1322,  ..., -3.7371, -1.15

tensor([[[-1.5423,  0.6047, -1.0909,  ...,  1.2873, -1.2296, -0.0078],
         [ 1.7750,  0.5830, -0.6324,  ..., -0.2918,  0.5648,  2.4999],
         [-0.4109,  0.3604, -0.5681,  ..., -1.3533, -1.5468, -0.6411],
         [-0.5984, -1.1898,  1.1540,  ...,  0.3576, -0.5583, -0.8304]],

        [[-0.5086,  0.1581, -0.3555,  ..., -1.2080, -1.3556, -0.5454],
         [-0.7371, -2.8679,  0.7941,  ...,  0.0880, -1.4434, -0.8796],
         [-0.6068, -0.1514, -1.0054,  ..., -0.3300,  1.4072,  0.3914],
         [-1.5833,  0.5747, -1.0992,  ...,  1.1237, -1.1704,  0.4744]],

        [[-1.3489,  0.6143, -0.8064,  ...,  0.8160,  0.0677, -0.7267],
         [-0.1931,  0.5949, -0.2865,  ..., -1.8183,  0.8409,  0.9704],
         [-0.7976, -0.2078,  3.4072,  ..., -2.3644, -1.8277,  0.3987],
         [ 1.2767, -0.2212,  0.9691,  ...,  0.1053,  1.1661,  2.7814]],

        ...,

        [[-0.0994, -2.2927,  0.8742,  ...,  0.3574, -0.9617, -1.4962],
         [-0.7294,  0.1987, -0.6578,  ..., -0.5753,  1.35

tensor([[[-0.5385, -1.2128,  0.8765,  ...,  0.3783, -0.4929, -1.0426],
         [-1.0226,  0.3671, -0.9846,  ..., -0.1765,  1.4004,  0.5500],
         [-1.6236,  0.6459, -1.0691,  ...,  1.1492, -1.0601,  0.6016],
         [ 1.6795,  0.6187, -0.5446,  ..., -0.5066,  0.6072,  2.5119]],

        [[-1.5705,  0.5883, -1.0936,  ...,  1.0680, -1.3858,  0.4213],
         [ 1.4081,  0.5557, -0.6205,  ..., -0.4671,  0.5791,  2.3948],
         [-0.3824, -0.0549, -0.4191,  ..., -1.0658, -1.6882, -0.9832],
         [-1.2926,  0.5936, -1.0477,  ..., -1.5958,  1.3296,  0.0058]],

        [[ 0.1302, -2.3525,  3.0478,  ..., -0.0123, -1.4490, -1.5142],
         [-0.5245, -0.0880, -0.4152,  ..., -0.7463,  1.3125, -0.4227],
         [-1.3682,  0.6276, -0.5652,  ...,  0.7577,  0.1386, -0.4145],
         [-0.3202,  0.6061, -0.1158,  ..., -1.9556,  0.8491,  0.9600]],

        ...,

        [[ 0.0752, -2.3482,  0.3591,  ...,  0.4959, -0.0980, -1.5253],
         [-0.7278,  0.1390, -0.8491,  ..., -0.3298,  1.39

tensor([[[-0.1907, -0.1899, -0.9704,  ..., -0.0574,  1.3991,  0.3351],
         [-1.5271,  0.5891, -1.0749,  ...,  1.2697, -1.1722,  0.3476],
         [ 1.4104,  0.5660, -0.6510,  ..., -0.3323,  0.6034,  2.3332],
         [-0.4371,  0.2351, -0.4860,  ..., -1.3509, -1.1956, -0.0848]],

        [[ 1.2216,  0.5643, -0.6424,  ..., -0.6706,  0.8010,  1.9545],
         [-0.4057, -0.0479, -0.4873,  ..., -1.2317, -1.4481, -1.2258],
         [-0.5981, -3.4950,  0.4935,  ...,  0.0921, -1.5896, -1.4587],
         [-0.7949, -0.1221, -1.0159,  ..., -0.3584,  1.4105, -0.2404]],

        [[-1.4559,  0.6244, -0.6650,  ...,  0.9696,  0.3663, -0.9953],
         [-1.0659,  0.5969, -0.2535,  ..., -1.8838,  0.8099,  0.4602],
         [-1.2903, -0.0997,  3.6368,  ..., -2.2242, -1.3762,  0.1383],
         [ 0.3407, -0.1205,  1.0783,  ..., -0.1568,  1.1254,  2.4552]],

        ...,

        [[ 0.4234,  0.0416, -0.2000,  ..., -1.3819, -1.0605, -1.6336],
         [ 0.2982, -1.7327,  1.2514,  ...,  0.3565, -0.45

In [132]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,1.072231,2.046041,0.512025,00:03


tensor([[[-2.2714e-02, -3.3153e-02,  1.4351e-02,  ...,  6.0412e-02,
           3.6438e-03, -2.4120e-02],
         [ 9.1147e-02, -7.2128e-02, -2.2644e-03,  ..., -7.1827e-02,
          -6.8440e-02, -1.3795e-01],
         [ 9.5430e-02, -9.2424e-02, -1.1322e-01,  ..., -7.1682e-02,
          -9.2116e-02, -1.8097e-01],
         [ 1.8725e-01, -4.7784e-02, -1.5822e-01,  ..., -5.7868e-02,
          -2.3237e-02, -1.2781e-01]],

        [[ 1.8513e-02, -1.0711e-01,  1.0311e-01,  ...,  7.8635e-02,
          -3.4928e-02, -3.4165e-02],
         [ 8.1088e-02, -4.4308e-02, -1.2346e-03,  ...,  4.2223e-02,
          -3.5905e-03, -1.2816e-01],
         [ 4.5858e-02, -3.8705e-02, -9.4193e-02,  ...,  5.4225e-02,
          -4.0199e-03, -1.7080e-01],
         [ 1.4382e-01,  1.0676e-04, -1.2220e-01,  ...,  3.6867e-02,
           4.3592e-02, -1.1761e-01]],

        [[-1.5789e-02,  7.2668e-02, -8.7435e-03,  ...,  8.1686e-02,
          -3.4306e-02,  1.7123e-02],
         [-3.7906e-02,  5.7868e-02, -8.8451e-02,  .

tensor([[[-0.0216,  0.0709, -0.1171,  ...,  0.0817, -0.1634, -0.1866],
         [-0.0237, -0.0190, -0.0715,  ...,  0.1324, -0.1833, -0.2357],
         [-0.0629, -0.0703, -0.0899,  ...,  0.1515, -0.1603, -0.2483],
         [ 0.0283,  0.0324, -0.1727,  ...,  0.0946, -0.0860, -0.2862]],

        [[ 0.0524, -0.2118,  0.0155,  ...,  0.0838, -0.0218, -0.0845],
         [ 0.0572, -0.1725, -0.0054,  ...,  0.0443, -0.0283, -0.1806],
         [-0.0114, -0.1523, -0.0742,  ...,  0.0551, -0.0279, -0.2004],
         [ 0.0945, -0.0878, -0.1216,  ...,  0.0539,  0.0286, -0.1415]],

        [[-0.0269, -0.0925, -0.0822,  ..., -0.0909, -0.1499, -0.2845],
         [ 0.0309, -0.0361, -0.1114,  ..., -0.0732, -0.0444, -0.2709],
         [ 0.0117, -0.1019,  0.0305,  ...,  0.0491, -0.0353, -0.2463],
         [ 0.0117, -0.1045,  0.0580,  ..., -0.0834, -0.0682, -0.2854]],

        ...,

        [[-0.0036,  0.0521,  0.1282,  ...,  0.2821, -0.0549, -0.1788],
         [-0.0241,  0.0567,  0.0670,  ...,  0.1934, -0.01

tensor([[[-0.1490,  0.0314, -0.0052,  ..., -0.1681, -0.0456, -0.2536],
         [-0.1353,  0.0187, -0.0202,  ..., -0.1853, -0.1478, -0.2287],
         [-0.1455,  0.1033, -0.0619,  ..., -0.0608, -0.1582, -0.1575],
         [-0.2147,  0.0513, -0.0928,  ..., -0.0558, -0.0680, -0.1639]],

        [[-0.0915, -0.1573, -0.0510,  ..., -0.0251,  0.0221, -0.1110],
         [ 0.0083, -0.1237, -0.1116,  ..., -0.0057,  0.0710, -0.0923],
         [-0.0006, -0.2075,  0.0678,  ...,  0.1296,  0.0444, -0.1202],
         [-0.0751, -0.1667,  0.0834,  ...,  0.0531, -0.0943, -0.1025]],

        [[-0.0966, -0.0813,  0.0706,  ...,  0.2072,  0.1099, -0.2393],
         [-0.0992, -0.1152, -0.0340,  ...,  0.1360,  0.0584, -0.2785],
         [-0.0554, -0.0654, -0.0707,  ...,  0.2268, -0.0797, -0.2455],
         [-0.1321, -0.0760, -0.0624,  ...,  0.1537, -0.0097, -0.2608]],

        ...,

        [[-0.1314,  0.0310,  0.0495,  ...,  0.0922, -0.1107, -0.2701],
         [-0.1489,  0.0801,  0.1349,  ..., -0.0051, -0.09

tensor([[[-0.2898, -0.4082, -0.1381,  ..., -0.2296, -0.0734, -0.3148],
         [-0.3082, -0.4269, -0.2062,  ..., -0.2256, -0.1029, -0.2320],
         [-0.1355, -0.3316, -0.2282,  ..., -0.1053, -0.0716, -0.1958],
         [-0.2161, -0.3439, -0.1662,  ..., -0.1111,  0.0595, -0.2432]],

        [[-0.2556, -0.0657,  0.0296,  ...,  0.0633, -0.0344, -0.1775],
         [-0.1471, -0.0542, -0.0741,  ...,  0.0683,  0.0502, -0.1905],
         [-0.1315, -0.1592,  0.1129,  ...,  0.2389,  0.0331, -0.2178],
         [-0.1516, -0.1219,  0.1260,  ...,  0.1145, -0.1564, -0.1654]],

        [[-0.2287, -0.3102,  0.1775,  ...,  0.2648,  0.0524, -0.3448],
         [-0.1975, -0.2741,  0.1736,  ...,  0.1677,  0.0503, -0.2204],
         [-0.3152, -0.3223,  0.0956,  ...,  0.0154,  0.1537, -0.2354],
         [-0.2568, -0.2689,  0.0011,  ..., -0.0045,  0.1769, -0.2750]],

        ...,

        [[-0.1710, -0.2919,  0.0788,  ..., -0.2636,  0.0228, -0.3575],
         [-0.2213, -0.2319,  0.0641,  ..., -0.2471,  0.01

...

tensor([[[ 0.0129,  0.6368,  0.0097,  ...,  0.0029,  0.0021, -0.6442],
         [-0.0146,  0.4248,  0.2985,  ...,  0.0462, -0.0953, -0.5096],
         [-0.0323,  0.1656,  0.5044,  ..., -0.0667, -0.7442, -0.3665],
         [ 0.0289,  0.6150,  0.6776,  ..., -0.0827, -0.4417, -0.2743]],

        [[-0.2209,  0.0319,  0.5033,  ..., -0.0233, -0.7798, -0.5633],
         [-0.0698,  0.6814,  0.7658,  ..., -0.1020, -0.6770, -0.4613],
         [-0.2562, -0.5107,  0.6817,  ...,  0.0011, -0.1308, -0.1645],
         [-0.1795,  0.5115, -0.0744,  ...,  0.0346, -0.0456, -0.7286]],

        [[-0.5763,  0.1004, -0.2717,  ...,  0.2511, -0.4193, -0.7892],
         [-0.5661, -0.0544, -0.3613,  ...,  0.2665, -0.7090, -0.2800],
         [-0.5001,  0.3894, -0.2311,  ...,  0.1188, -0.5586, -0.4352],
         [-0.5163,  0.6138,  0.2466,  ...,  0.2620, -0.5798, -0.3810]],

        ...,

        [[-0.4891,  0.1042, -0.4334,  ..., -0.1772, -0.5962, -0.1282],
         [-0.3608,  0.7563,  0.0440,  ..., -0.2872, -0.47

tensor([[[-0.0535,  0.3226,  0.3666,  ...,  0.1138,  0.2176, -0.7098],
         [-0.0788,  0.1785,  0.6339,  ...,  0.0233, -0.6429, -0.6404],
         [ 0.0924,  0.7143,  0.8036,  ..., -0.0314, -0.7379, -0.5345],
         [-0.0944, -0.5612,  0.7011,  ...,  0.0447, -0.1983, -0.2810]],

        [[-0.1937,  0.7229,  0.7769,  ..., -0.1123, -0.7911, -0.4225],
         [-0.3440, -0.4703,  0.6875,  ..., -0.0397, -0.1221, -0.1729],
         [-0.2483,  0.5501, -0.0762,  ..., -0.0592, -0.0181, -0.7701],
         [-0.2941,  0.3543,  0.1975,  ..., -0.1309, -0.1708, -0.5623]],

        [[-0.3648,  0.4781, -0.1361,  ..., -0.0515, -0.4646, -0.3921],
         [-0.5551,  0.6497,  0.3269,  ...,  0.1588, -0.5352, -0.3405],
         [-0.1673,  0.5331,  0.5267,  ...,  0.0764, -0.7507, -0.1381],
         [-0.5203, -0.4223,  0.3576,  ...,  0.1501, -0.3088, -0.2096]],

        ...,

        [[-0.4660, -0.0212, -0.3968,  ...,  0.0354, -0.5711, -0.2332],
         [-0.3266,  0.7129, -0.0028,  ..., -0.1075, -0.41

tensor([[[-8.5439e-02,  2.0331e-01,  6.7774e-01,  ..., -3.6298e-02,
          -5.8689e-01, -5.4819e-01],
         [-2.0618e-02,  2.8524e-01,  7.8016e-01,  ..., -7.4590e-02,
          -6.7218e-01, -3.0423e-01],
         [ 3.7773e-02, -6.6407e-01,  6.8908e-01,  ...,  2.6264e-02,
          -7.0599e-03, -3.2580e-01],
         [ 9.1418e-02,  3.6763e-01,  6.7019e-02,  ...,  4.9923e-02,
           3.9042e-01, -8.2981e-01]],

        [[-2.3680e-01, -5.4025e-01,  6.0895e-01,  ...,  1.0696e-02,
           3.7658e-02, -2.7428e-01],
         [-1.4778e-01,  5.5105e-01, -3.0593e-02,  ..., -1.4627e-04,
           1.5469e-01, -7.7963e-01],
         [-2.0668e-01,  3.9798e-01,  2.5897e-01,  ..., -7.6765e-02,
          -1.1713e-01, -5.8756e-01],
         [-2.1970e-01,  1.7619e-01,  5.6590e-01,  ..., -2.0963e-01,
          -7.6416e-01, -4.5939e-01]],

        [[-5.5521e-01, -9.1765e-02, -4.2031e-01,  ...,  1.8948e-01,
          -2.9534e-01, -8.1152e-01],
         [-5.5407e-01, -2.0700e-01, -5.0842e-01,  .

tensor([[[-0.1162,  0.1590,  0.5888,  ...,  0.0165, -0.6631, -0.6595],
         [ 0.0100,  0.7192,  0.7794,  ..., -0.0510, -0.7114, -0.5334],
         [ 0.0104, -0.5258,  0.7329,  ...,  0.0045, -0.2006, -0.2314],
         [ 0.0773,  0.5575,  0.1043,  ...,  0.0052,  0.1886, -0.7402]],

        [[-0.1976,  0.4363,  0.2738,  ..., -0.0127,  0.1491, -0.7797],
         [-0.2472,  0.2821,  0.4829,  ...,  0.1877, -0.1654, -0.6958],
         [-0.2366, -0.0518,  0.6294,  ...,  0.0143, -0.7376, -0.6018],
         [-0.1629,  0.4573,  0.7796,  ..., -0.0629, -0.4990, -0.4535]],

        [[-0.4670,  0.4751, -0.2103,  ..., -0.0191, -0.5582, -0.2388],
         [-0.5360,  0.6617,  0.2517,  ...,  0.1827, -0.6325, -0.1746],
         [-0.2756,  0.6378,  0.4682,  ...,  0.1336, -0.7722, -0.0399],
         [-0.5816, -0.3205,  0.3631,  ...,  0.1610, -0.4715, -0.1421]],

        ...,

        [[-0.3045,  0.7320,  0.0580,  ..., -0.4515, -0.3870, -0.5954],
         [ 0.0648,  0.7972,  0.4494,  ..., -0.0705, -0.49

tensor([[[-8.9300e-03,  7.5511e-01,  7.8993e-01,  ...,  5.2236e-02,
          -6.5331e-01, -4.1658e-01],
         [-9.2260e-02, -5.0232e-01,  7.1771e-01,  ...,  9.5811e-02,
          -7.6513e-02, -1.4881e-01],
         [-2.3301e-02,  5.7482e-01,  1.3625e-03,  ...,  5.8107e-02,
           2.0736e-01, -7.5397e-01],
         [-8.6820e-02,  3.7317e-01,  3.2526e-01,  ...,  3.3630e-03,
           1.3545e-01, -6.3424e-01]],

        [[-1.9941e-01,  5.5941e-01,  2.8370e-01,  ..., -1.0839e-01,
           1.3572e-01, -6.4994e-01],
         [-2.6151e-01,  3.7453e-01,  4.7635e-01,  ...,  2.0087e-03,
          -7.0115e-02, -5.7015e-01],
         [-2.5984e-01, -6.0049e-04,  6.1768e-01,  ..., -1.5467e-01,
          -6.8425e-01, -4.6854e-01],
         [-1.6499e-01,  2.2719e-01,  7.7270e-01,  ..., -2.0102e-01,
          -7.2044e-01, -2.4943e-01]],

        [[-5.1937e-01,  3.2981e-01,  4.2594e-01,  ...,  1.4144e-01,
          -7.9612e-01, -1.6022e-01],
         [-4.4271e-01,  6.1621e-01,  5.2520e-01,  .

## fin